<a href="https://colab.research.google.com/github/CRAFTYPROGRAMMER826/IITGN-SRIP2025-IP0NB0000020/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tsfel
import numpy as np
import pandas as pd
import os
import tsfel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, Dense, Flatten, Dropout, MaxPooling1D

# installing tsfel library and importing the necessary packages and modules


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.9 MB/s eta 0:00:00


In [1]:
!wget -O dataset.zip "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"
!unzip "dataset.zip" -d "/content/"
!unzip "UCI HAR Dataset.zip" -d "/content/UCI HAR"

--2025-03-01 08:23:47--  https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataset.zip’

dataset.zip             [       <=>          ]  58.18M  40.6MB/s    in 1.4s    

2025-03-01 08:23:49 (40.6 MB/s) - ‘dataset.zip’ saved [61005872]

Archive:  dataset.zip
 extracting: /content/UCI HAR Dataset.names  
 extracting: /content/UCI HAR Dataset.zip  
Archive:  UCI HAR Dataset.zip
   creating: /content/UCI HAR/UCI HAR Dataset/
  inflating: /content/UCI HAR/UCI HAR Dataset/.DS_Store  
   creating: /content/UCI HAR/__MACOSX/
   creating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/
  inflating: /content/UCI HAR/__MACOSX/UCI HAR Dataset/._.DS_Store  
  inflating: /content/UCI HAR/UCI HAR Dataset/activity_labels.txt  


In [3]:
# Define paths
train_path = "UCI HAR/Dataset/train/Inertial Signals/"
test_path = "UCI HAR/Dataset/test/Inertial Signals/"

x_train = np.dstack([
    np.loadtxt(train_path + f"body_acc_x_train.txt"),
    np.loadtxt(train_path + f"body_acc_y_train.txt"),
    np.loadtxt(train_path + f"body_acc_z_train.txt"),
    np.loadtxt(train_path + f"body_gyro_x_train.txt"),
    np.loadtxt(train_path + f"body_gyro_y_train.txt"),
    np.loadtxt(train_path + f"body_gyro_z_train.txt"),
    np.loadtxt(train_path + f"total_acc_x_train.txt"),
    np.loadtxt(train_path + f"total_acc_y_train.txt"),
    np.loadtxt(train_path + f"total_acc_z_train.txt"),
])

# Load testing data (Body Acceleration, Gyroscope, and Total Acceleration)
x_test = np.dstack([
    np.loadtxt(test_path + f"body_acc_x_test.txt"),
    np.loadtxt(test_path + f"body_acc_y_test.txt"),
    np.loadtxt(test_path + f"body_acc_z_test.txt"),
    np.loadtxt(test_path + f"body_gyro_x_test.txt"),
    np.loadtxt(test_path + f"body_gyro_y_test.txt"),
    np.loadtxt(test_path + f"body_gyro_z_test.txt"),
    np.loadtxt(test_path + f"total_acc_x_test.txt"),
    np.loadtxt(test_path + f"total_acc_y_test.txt"),
    np.loadtxt(test_path + f"total_acc_z_test.txt"),
])

y_train = np.loadtxt("UCI HAR/Dataset/train/y_train.txt")
y_test = np.loadtxt("UCI HAR/Dataset/test/y_test.txt")



In [4]:
print("Train Data Shape:", x_train.shape)
print("Test Data Shape:", x_test.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (7352, 128, 9)
Test Data Shape: (2947, 128, 9)
Train Labels Shape: (7352,)
Test Labels Shape: (2947,)


In [5]:
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(y_train - 1)  # Subtract 1 to make labels zero-indexed
y_test_categorical = to_categorical(y_test - 1)

print(y_train_categorical.shape, y_test_categorical.shape)  # Should be (7352, 6) and (2947, 6)


(7352, 6) (2947, 6)


In [6]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

print(x_train_flat.shape, x_test_flat.shape)  # Should be (7352, 1152) and (2947, 1152)


(7352, 1152) (2947, 1152)


In [7]:
from sklearn.preprocessing import StandardScaler

# Check data statistics before scaling
print("Before Scaling:")
print("Mean:", x_train_flat.mean(), "Std:", x_train_flat.std())

# Apply Standard Scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)

# Check data statistics after scaling
print("\nAfter Scaling:")
print("Mean:", x_train_scaled.mean(), "Std:", x_train_scaled.std())



Before Scaling:
Mean: 0.10206605723804434 Std: 0.4021651763827932

After Scaling:
Mean: -3.362481775705498e-17 Std: 0.9999999999999998


In [11]:
# Import Logistic Regression
from sklearn.linear_model import LogisticRegression

# Initialize the model
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
log_reg.fit(x_train_scaled, y_train)

# Make predictions
y_pred = log_reg.predict(x_test_scaled)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")




Accuracy: 0.5786
